In [1]:
import pandas
import requests
from multiprocessing import Process, Queue, Pool


length = 50
total_length = 500

group_items = {}

url = 'https://www.cupoy.com/MixNewsMongoAction.do'

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
def get_articles(start_news_id, length):
    
    items = pandas.DataFrame()
    payload = {'op': 'getTopMixNews', 'nationid': 'TW', 'startNewsID': start_news_id, 'len': length}

    try:
        count = 0
        response = requests.get(url, params=payload).json()
        for item in response['items']:
            article_id = start_news_id + count
            item_df = pandas.DataFrame({
                'title': item.get('title'),
                'description': item.get('description'),
                'linkurl': item.get('linkurl'),
                'keywordtrend': item['metrics']['keywordtrend'],
                'uidactscore': item['metrics']['uidactscore'],
                'socialactscore': item['metrics']['socialactscore'],
                'activescore': item['metrics']['activescore'],
                'groups': (str(item['bucketgrpids']) if 'bucketgrpids' in item else None)
            }, index=[article_id])

            print(article_id, item['title'], sep='\t')
            items = items.append(item_df)
            count +=1
        return items
    except Exception as err:
        print(err)
        return



In [3]:
pqueue = Queue()
items = pandas.DataFrame()
with Pool() as pool:

    results = [pool.apply_async(get_articles, (start_news_id, length)) for start_news_id in range(0, total_length, length)]
    
    for result in results:
        items = items.append(result.get())




 little d
146	新機難產...Android 之父 Andy Rubin 的手機新創 Essential 宣佈倒閉 - INSIDE
150	兩岸喬不攏「臺胞撤離包機」　北京「要面子」害慘湖北臺灣人
38	'Parasite' Makes Another Leap for South Korea's Cultural Economy  - The News Lens International Edition
147	每日一句學管理 | 經理人
151	「鑽石公主號」乘客可以下船了！羣聚感染太嚴重，日本政府終於同意「老病者明天上岸」-風傳媒
39	生醫發展第一線的無名英雄— 實驗小鼠的使用與倫理 - 科學月刊Science Monthly
152	湖北一把手被拔官！武漢肺炎撼動中共人事：省委書記蔣超良去職，上海市委書記應勇調任救火-風傳媒
148	【ClubLike 口罩】2月14 發售 1,000 盒藍鷹牌兒童口罩, 教你先攞 HK$50 優惠碼 | Qooah
153	疫情大爆發？中國更改確診標準，武漢肺炎患者一口氣新增1萬5千人、252死-風傳媒
40	因應「武漢肺炎」國際活動相繼取消或延期，東京奧運呢？ - The News Lens 關鍵評論網
149	京都住宿新選擇！恐怖房、求神問卜房，超狂的概念旅館「不思議な宿」 | MATCHA - 日本線上旅遊觀光雜誌
41	微軟新創加速器第二期招募
154	Slack最大客戶IBM，敲定35萬名員工全都採用Slack溝通協作｜數位時代
155	特斯拉要自產電池？低調收購兩家技術商後，傳正打造首條自家生產線｜數位時代
42	哪裡是人類的起源？ 眾說紛紜的單地、多地論 - 科學月刊Science Monthly
156	電子書閱讀器mooInk Pro 13吋版本現身！限量千臺集資，若未達目標將不生產｜數位時代
43	新訂一年方案：《科學月刊》一年12期 - 科學月刊Science Monthly
157	M.2 NVMe SSD 隨意熱插拔，ICY DOCK 推出 ToughArmor MB720M2K-B 4 槽 5.25 吋抽取盒 | T客邦
44	不滿柬埔寨人權無改善，歐盟撤回柬埔寨部分零關稅優惠 - The News Lens 關鍵評論網
158	在丹波篠山的3天2夜之旅ーー鄰近大阪・京都的城下

In [4]:
items.to_csv('topnews.csv')
items

,title,description,linkurl,keywordtrend,uidactscore,socialactscore,activescore,groups
0,尺寸更大、讀寫體驗升級！Readmoo mooInk Pro 13 搶先試用心得分享 | T...,Readmoo 讀墨電子書最新 13.3 吋電子書閱讀器「mooInk Pro 13」，以重...,https://technews.tw/2020/02/13/readmoo-mooink-...,0,85,0,34272,"['3CExpert_tw', 'tech_tw']"
1,科學家在「量子糾纏」取得重大突破，「量子網際網路」時代將提前到來 | TechNews 科技新報,科學家們已經成功將兩個距離比以往更遠的「量子記憶體」（Quantum Memory）糾纏在一...,https://technews.tw/2020/02/13/scientists-make...,0,85,0,34272,"['3CExpert_tw', 'tech_tw']"
2,中國徵收私產，癌末病患被踢出院！傳蘇州工廠開工爆疫情 | TechNews 科技新報,武漢肺炎的確診患者不斷增加，讓中國醫療系體系瀕臨臨界點，中國當局開始緊急徵收民營醫院、旅館、...,https://technews.tw/2020/02/13/china-wuhan-pne...,0,85,0,34272,"['3CExpert_tw', 'tech_tw']"
3,波士頓動力機器狗要上工了，成為鑽油平臺的巡守員 | TechNews 科技新報,波士頓動力公司能開門、爬梯、跳舞、後空翻的機器狗 Spot，剛剛找到了第一份正式工作。\r\...,https://technews.tw/2020/02/13/boston-dynamics...,0,85,0,34272,"['3CExpert_tw', 'tech_tw']"
4,中國半導體業現況：晶圓廠正常運轉，晶片設計遠端辦公 | TechNews 科技新報,新浪科技引述第一財經報導，中國新冠肺炎疫情嚴峻，半導體業受此次疫情的影響程度也引發關注；據瞭...,https://technews.tw/2020/02/13/china-semicondu...,0,85,0,34272,"['3CExpert_tw', 'tech_tw']"
...,...,...,...,...,...,...,...,...
495,接下來的疫情關注三件事：「武漢肺炎」的特性、醫療資源侷限性，以及臺灣防疫具體目標 - The...,現階段臺灣沒社區感染，正確嚴格的隔離政策最重要，其次才是個人防疫。只要境外的病毒不進來，境內...,https://www.thenewslens.com/article/131104,0,35,0,3388,None
496,對中國一廂情願的「非洲秀才」，成為被中國利用的棋子 - The News Lens 關鍵評論網,譚德塞抗疫的宗旨是「別讓中國不開心」，但病毒卻很開心，防線不知道何時會被突破的各國，也愈來愈...,https://www.thenewslens.com/article/131155,0,35,0,3388,"['WhiteCollar_tw', 'business_tw']"
497,將近一年的選舉廝殺後，能夠療癒社會的政治領袖存在嗎？ - The News Lens 關鍵評論網,如果我們期望更成熟的公民社會，我們無法不從意見領袖自身的內在關照與心理反思出發，而意見領袖對...,https://www.thenewslens.com/article/131060,0,35,0,3388,None
498,梅克爾的基民盟會不會跟著另類選擇黨「右傾」？ - The News Lens 關鍵評論網,曾被普遍視為梅克爾接班人的克朗普-卡倫鮑爾打算辭去基民盟主席職務。這一發展是否意味著該黨將更...,https://www.thenewslens.com/article/128955,0,35,0,3388,"['InternationalFinance_tw', 'business_tw']"


In [5]:
url = 'https://www.cupoy.com/MixNewsAction.do?op=getBucketGroups&nationid=TW&len=50'
response = requests.get(url)

group_map = {}
for group in response.json():
    group_map[group['groupid']] = group['name']

group_items = {}
for i in range(len(items)):
    item_groups = items.iloc[i]['groups']
    if item_groups:
        item_groups = item_groups.replace('[', '').replace(']', '').replace('\'', '').split(', ')
        for group in item_groups:
            group = group_map[group]
            if group not in group_items:
                group_items[group] = [i]
            else:
                group_items[group].append(i)

for groupid in sorted(group_items, key=lambda k: len(group_items[k]), reverse=True):
    print(groupid, len(group_items[groupid]))


科技 98
商業 90
生活 71
職場白領 63
御宅學園 57
運動 47
享樂女性 46
親子家庭 42
國際財經 42
3C達人 38
遊戲 38
文青聚落 23
健身瘦身 21
城市食旅 16
設計 10


In [6]:

print('Trend News: ')
print('From keywordtrend', items.loc[items['keywordtrend'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())
print('From uidactscore', items.loc[items['uidactscore'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())
print('From socialactscore', items.loc[items['socialactscore'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())
print('From activescore', items.loc[items['activescore'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())


Trend News: 
From keywordtrend {'title': '尺寸更大、讀寫體驗升級！Readmoo mooInk Pro 13 搶先試用心得分享 | TechNews 科技新報', 'keywordtrend': 0, 'uidactscore': 85, 'socialactscore': 0, 'activescore': 34272}
From uidactscore {'title': '兩大廠青睞！ITM國際信任機器拿下高通創新競賽亞軍後，再獲聯發科Pre-A輪投資｜數位時代', 'keywordtrend': 0, 'uidactscore': 105, 'socialactscore': 0, 'activescore': 10164}
From socialactscore {'title': 'NOWnews 今日新聞 | 先補貼後調價？燃油機車價格漲\u3000網友：油車廠商賺飽飽？', 'keywordtrend': 0, 'uidactscore': 80, 'socialactscore': 36934, 'activescore': 7232}
From activescore {'title': '尺寸更大、讀寫體驗升級！Readmoo mooInk Pro 13 搶先試用心得分享 | TechNews 科技新報', 'keywordtrend': 0, 'uidactscore': 85, 'socialactscore': 0, 'activescore': 34272}
